In [ ]:
zero_frac = 0.25
correction = 0.2
top_crop, bottom_crop, left_crop, right_crop = 60, 25, 0, 0
EPOCHS = 5

## Build Log

In [ ]:
import pandas as pd
%matplotlib inline

In [ ]:
driving_log = pd.read_csv('data/driving_log.csv', nrows=10)#, usecols=['center', 'left', 'right', 'steering']) #['throttle', 'brake', 'speed']
driving_log.head()

In [ ]:
driving_log['steering'].describe()

In [ ]:
_ = driving_log['steering'].hist(bins=100, figsize=(15,5))

In [ ]:
def undersample_zero_steerings(log, frac):
    log_nonzero = log[log['steering']!=0]
    log_zero = log[log['steering']==0].sample(frac=frac)
    log = log_zero.append(log_nonzero)
    log = log.reset_index(drop=True)
    return log

In [ ]:
driving_log = undersample_zero_steerings(driving_log, zero_frac)

In [ ]:
driving_log['steering'].describe()

In [ ]:
_ = driving_log['steering'].hist(bins=100, figsize=(15,5))

In [ ]:
def augment_left_right(driving_log, correction):
    log = pd.DataFrame({'image':driving_log['center']
                                .append(driving_log['left'])
                                .append(driving_log['right']),
                        'steering':driving_log['steering']
                                .append(driving_log['steering'] + correction)
                                .append(driving_log['steering'] - correction)})

    log = log[(log['steering']>=-1)&(log['steering']<=1)]
    return log

In [ ]:
log = augment_left_right(driving_log, correction)

In [ ]:
log['steering'].describe()

In [ ]:
_ = log['steering'].hist(bins=100, figsize=(15,5))

In [ ]:
log = log.sample(frac=1).reset_index(drop=True)  #shuffle log DataFrame

## Build Training and Validation Data

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers.convolutional import Convolution2D

from sklearn.model_selection import train_test_split
import sklearn

%matplotlib inline

In [ ]:
log.head()

In [ ]:
train_log, validation_log = train_test_split(log, test_size=0.2)

train_log = train_log.reset_index(drop=True)
validation_log = validation_log.reset_index(drop=True)

In [ ]:
def preprocess(image):
    image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)[:,:,1]
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image = image[:,:,None]
    
    return image

In [ ]:
def get_data(log):    
    images = []
    steerings = []
    
    for _, row in log.iterrows():
        filename = 'data/'+row['image'].strip()
        image = preprocess(mpimg.imread(filename))
        image_flip = cv2.flip(image, 1)[:,:,None]
        steering = row['steering']

        images.extend([image, image_flip])
        steerings.extend([steering, -steering])

    X_train, y_train = np.array(images), np.array(steerings)
    X_train, y_train = sklearn.utils.shuffle(X_train, y_train)
    
    return X_train, y_train

In [ ]:
log.shape, train_log.shape, validation_log.shape

In [ ]:
def generator(sample_log, batch_size=32):
    n_rows = len(sample_log)
    while 1: # Loop forever so the generator never terminates
        sample_log = sample_log.sample(frac=1).reset_index(drop=True)  #shuffle sample_log DataFrame
        
        for offset in range(0, n_rows, batch_size):
            batch_log = sample_log[offset : offset + batch_size]

            X, y = get_data(batch_log)
            
            yield X, y 

In [ ]:
train_generator = generator(train_log, batch_size=128)
validation_generator = generator(validation_log, batch_size=128)

## Compile and train the model

In [ ]:
model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,1)))
model.add(Cropping2D(cropping=((top_crop, bottom_crop), (left_crop, right_crop)), input_shape=(160,320,1)))

model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))
model.add(Convolution2D(64,3,3, activation='relu'))

model.add(Flatten())

model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))

model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
#history_object = model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=2) 

history_object = model.fit_generator(train_generator,
                                     samples_per_epoch = len(train_log),
                                     validation_data = validation_generator, 
                                     nb_val_samples=len(validation_log),
                                     nb_epoch=EPOCHS,
                                     verbose=1)

In [ ]:
model.save('model.h5')

In [ ]:
### print the keys contained in the history object
print(history_object.history.keys())
### plot the training and validation loss for each epoch

plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')